In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import metrics, preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn import svm

%matplotlib inline

In [2]:
ja_tokenizer = Tokenizer()
def ja_tokenize(text):
    res=[]
    lines=text.split('\n')
    for line in lines:
        malist = ja_tokenizer.tokenize(line)
        for tok in malist:
            ps=tok.part_of_speech.split(",")[0]
            if not ps in ['名詞', '動詞', '形容詞', '形容動詞']: continue
            w=tok.base_form
            if w=="*" or w=="": w=tok.surface
            if w=="" or w=="\n": continue
            res.append(w)
        res.append("\n")
    return res

#未知データ用関数
def predicted_group(s, answer, model):
    pred = model.predict([s])
    return answer[pred[0]]

In [3]:
# a = pd.read_excel('training.xlsx', sheet_name = 'メインシート', encoding='shift-jis')
a = pd.read_excel('EI_learning_data.xlsx', sheet_name = 'MCOR', encoding='shift-jis')
b = a[['件名/問い合わせ内容（状況）','Level1']]
b['Answer'] = b['Level1'].map({'CAx':1, 'RCD':2, 'ハーネス':3, 'その他':4, '出図管理':5, 'VSS':6})
b = b.dropna(subset=['件名/問い合わせ内容（状況）'])
b = b.reset_index(drop = True)
b['Answer'] = b['Answer'].astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
model = make_pipeline(CountVectorizer(),MultinomialNB(alpha=0.1,fit_prior = 'True'))
model2 = make_pipeline(TfidfVectorizer(),MultinomialNB(alpha=0.1,fit_prior = 'True'))
model3 = make_pipeline(CountVectorizer(),svm.SVC(gamma='scale'))
model4 = make_pipeline(TfidfVectorizer(),svm.SVC(gamma='scale'))

In [ ]:
data_train, data_test, labels_train, labels_test = train_test_split(b['件名/問い合わせ内容（状況）'], b['Answer'], train_size=0.70, random_state=10)

cvs = cross_val_score(model, b['件名/問い合わせ内容（状況）'],b['Answer'],cv=5)
cvs2 = cross_val_score(model2, b['件名/問い合わせ内容（状況）'],b['Answer'],cv=5)
cvs3 = cross_val_score(model3, b['件名/問い合わせ内容（状況）'],b['Answer'],cv=5)
cvs4 = cross_val_score(model4, b['件名/問い合わせ内容（状況）'],b['Answer'],cv=5)

In [17]:
# clf = MultinomialNB(alpha = 0.1, fit_prior='True')
print(cvs,cvs.mean())
print(cvs2,cvs2.mean())
print(cvs3,cvs3.mean())
print(cvs4,cvs4.mean())


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


[0.81390135 0.82882883 0.77477477 0.8018018  0.75395034]
0.7946514178594674
[0.81165919 0.82882883 0.79054054 0.80405405 0.77200903]
0.8014183291187816
[0.80269058 0.81756757 0.78153153 0.8018018  0.81038375]
0.8027950462077744
[0.80493274 0.82207207 0.76576577 0.80855856 0.8013544 ]
0.8005367067256548


In [ ]:
model4.fit(data_train,labels_train)
b['Predict'] = model4.predict(b['件名/問い合わせ内容（状況）'])
b.to_csv('EI_HELPDESK_Classification_2.csv')

In [4]:
count_vectorizer = CountVectorizer(analyzer=ja_tokenize)
tfidf = TfidfVectorizer(analyzer=ja_tokenize)

In [7]:
mat = count_vectorizer.fit_transform(b['件名/問い合わせ内容（状況）'].astype(str))
s = mat.toarray().tolist()
b['CV'] = s

In [1]:
# #ベクトル化したデータを抽出
# mat = tfidf.fit_transform(b['件名/問い合わせ内容（状況）'].astype(str))
# s = mat.toarray().tolist()
# b['tfidf'] = s
# b.head()
# # b.to_csv('EI_vectorize_data.csv')

In [9]:
b = b.dropna()

In [11]:
model = make_pipeline(CountVectorizer(),MultinomialNB(alpha=0.1,fit_prior = 'True'))
model2 = make_pipeline(TfidfVectorizer(),MultinomialNB(alpha=0.1,fit_prior = 'True'))
model3 = make_pipeline(CountVectorizer(),svm.SVC(gamma='scale'))
model4 = make_pipeline(TfidfVectorizer(),svm.SVC(gamma='scale'))

model.fit(b['件名/問い合わせ内容（状況）'],b['Answer'])
model2.fit(b['件名/問い合わせ内容（状況）'],b['Answer'])
model3.fit(b['件名/問い合わせ内容（状況）'],b['Answer'])
model4.fit(b['件名/問い合わせ内容（状況）'],b['Answer'])

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [12]:
ans = predicted_group("""< 他部門のワークエリア利用申請について >

昨日の19時頃に他部門のワークエリア利用申請を申請したが、

まだワークエリアが利用できない状況。

対応がいつ終わるか教えて欲しい。

業務で必要なことなので、できるだけ早く対応して欲しい。

""",b['Level1'],model4)
print('この問題は' + ans + 'で起きています。')

この問題はCAxで起きています。


In [15]:
def Chat_Bot_Prot():    
    while True:
        message = input('どうかしましたか？')
        if (message == '' or message == 'なにもない' or message == 'とくにない' or message =='さようなら' or message == 'ばいばい'):
            print('そうですか、、、また何かあったら読んでくださいね・・・')
            break
        ans = predicted_group(message,b['Level1'],model4)
        print('この問題は'+ ans + 'に関する問題ですね')

        if ans == 'その他':
            """
            分類した結果により処理を変える
            備考：ここは関数化して分岐したほうが良い
            """
            print('ヘルプデスクに問い合わせるのがいいと思います。')
            print('ヘルプデスクを送りますか?')
    

In [16]:
Chat_Bot_Prot()

どうかしましたか？
そうですか、、、また何かあったら読んでくださいね・・・
